In [1]:
import sqlite3 as sql
import pandas as pd

In [2]:
conn = sql.connect('database_subquery.sqlite')
conn

In [3]:
def sql_query(q):
    return pd.read_sql_query(q,conn)

In [15]:
q= '''SELECT name FROM sqlite_master'''
sql_query(q)

,name


In [30]:
q ='''CREATE TABLE employees (
      emp_id char(5) Primary Key, 
      name varchar(20) NOT NULL, 
      department varchar(20) NOT NULL, 
      Salary INT(8) NOT NULL,
      manager_id INT(5)
); '''

conn.execute(q)
conn.commit() ### error because the query executed twice.

OperationalError: table employees already exists

In [31]:

q= '''SELECT name FROM sqlite_master'''
sql_query(q)

,name
0,employees
1,sqlite_autoindex_employees_1


In [32]:
q= '''INSERT INTO employees (emp_id, name, department, Salary ,manager_id)
VALUES (1,	'Alice' ,	'HR',	60000,	5),
       (2,	'Bob',	'Sales',	55000,	5),
       (3,	'Charlie' ,	'IT',	70000,	6),
       (4,	'David',	'IT', 	72000,	6),
       (5	, 'Emma',	'HR',	90000 ,	NULL),
       (6,	'Frank'  ,	'IT',	95000	, NULL),
       (7,	'Grace', 	'Sales' ,	58000 ,	5)
;'''
conn.execute(q)
conn.commit()

In [33]:
q= '''SELECT * FROM  employees '''
sql_query(q)

,emp_id,name,department,Salary,manager_id
0,1,Alice,HR,60000,5.0
1,2,Bob,Sales,55000,5.0
2,3,Charlie,IT,70000,6.0
3,4,David,IT,72000,6.0
4,5,Emma,HR,90000,NaN
5,6,Frank,IT,95000,NaN
6,7,Grace,Sales,58000,5.0


### 1. Find employees who earn more than Alice.

In [34]:
q = ''' SELECT name 
FROM employees 
WHERE salary > (SELECT salary FROM employees WHERE name = 'Alice');
 '''
sql_query(q)

,name
0,Charlie
1,David
2,Emma
3,Frank


### 2. List employees who work in the same department as Emma or Frank.

In [35]:
q = ''' SELECT name 
FROM employees 
WHERE department IN (
    SELECT department FROM employees WHERE name IN ('Emma', 'Frank')
);
 '''
sql_query(q)

,name
0,Alice
1,Charlie
2,David
3,Emma
4,Frank


### 3. Find employees whose salary is greater than their department's average.

In [36]:
q = ''' SELECT name 
FROM employees e1
WHERE salary > (
    SELECT AVG(salary) 
    FROM employees e2 
    WHERE e2.department = e1.department
);
 '''
sql_query(q)

,name
0,Emma
1,Frank
2,Grace


### 4. Find the department with the highest paid employee.

In [37]:
q = '''SELECT department 
FROM employees 
WHERE salary = (SELECT MAX(salary) FROM employees);
  '''
sql_query(q)

,department
0,IT


### 5.List names of employees who are managers (i.e., their emp_id appears in manager_id).

In [38]:
q = '''SELECT name 
FROM employees 
WHERE emp_id IN (SELECT DISTINCT manager_id FROM employees WHERE manager_id IS NOT NULL);
'''
sql_query(q)

,name
0,Emma
1,Frank


### 6. Find employees who earn more than any other employee in their department (i.e., highest in their department).

In [44]:
q = '''
SELECT name 
FROM employees e1
WHERE NOT EXISTS (
    SELECT name 
    FROM employees e2 
    WHERE e2.department = e1.department
      AND e2.salary > e1.salary
)
'''
sql_query(q)


,name
0,Emma
1,Frank
2,Grace


### 7.Get the name of the lowest paid employee.

In [45]:
q = '''SELECT name 
FROM employees 
WHERE salary = (SELECT MIN(salary) FROM employees);
  '''
sql_query(q)

,name
0,Bob


### 8. Find employees who have the same salary as someone else.

In [47]:
q = '''
SELECT name 
FROM employees 
WHERE salary IN (
    SELECT salary 
    FROM employees 
    GROUP BY salary 
    HAVING COUNT(*) > 1
);
 '''
sql_query(q)   ### Result : because all the salaries are unique in the table.

,name


### 9.Show employees whose salary is less than their manager's salary.

In [49]:
q = ''' SELECT e1.name 
FROM employees e1 
WHERE salary < (
    SELECT e2.salary 
    FROM employees e2 
    WHERE e2.emp_id = e1.manager_id
);
 '''
sql_query(q)

,name
0,Alice
1,Bob
2,Charlie
3,David
4,Grace


### 10.List names of employees who earn equal to the average salary of their department.

In [50]:
q = ''' SELECT name 
FROM employees e1
WHERE salary = (
    SELECT AVG(salary) 
    FROM employees e2 
    WHERE e2.department = e1.department
);
 '''
sql_query(q)   #Result:None (no one’s salary exactly matches the average)

,name
